In [2]:
import pandas as pd

In [25]:
df = pd.read_csv('gender-classifier-DFE-791531.csv',encoding = "latin1")

In [26]:
df.head()

,Unnamed: 0,text
0,0,robbie e respond critic win eddie edward world...
1,1,felt friend i live story http retire http
2,2,absolutely louis start song hit hard feel good
3,3,hi jordanspieth look url use ifttt do typicall...
4,4,watch neighbour catch neighbs xxx xxx


In [8]:
df = df.drop(columns = ['Unnamed: 0'])

In [11]:
df.dtypes

text    object
dtype: object

In [13]:
df['text'] = df['text'].astype(str)

In [14]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import re

# Notice that these vectorizers are from `sklearn` and not `nltk`!
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
vec = CountVectorizer(min_df = 0.001, max_df = 0.80)
X = vec.fit_transform(df['text'])

# .get_feature_names() attribute useful
countvec_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
countvec_df


,able,absolutely,account,act,action,actually,add,address,adele,adult,...,ya,yeah,year,yes,yesterday,yet,yo,york,young,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
